# Day 2022-16: Proboscidea Volcanium

In [1]:
year = 2022
day  = 16

In [2]:
from local_settings import load_input
content = load_input(year, day)
print(f"[{content[:100]}...]")

Reading [https://adventofcode.com/2022/day/16/input]
3111 characters read.
[Valve EJ has flow rate=25; tunnel leads to valve MC
Valve WC has flow rate=0; tunnels lead to valves...]


# Part 1

In [3]:
# definitions for first part of problem solution
from collections import deque

def parseInp(s):
    valveMap = dict()
    workingValves = dict()
    for line in s.splitlines():
        tokens = line.split()
        valve = tokens[1]
        rate = int(tokens[4][5:-1])
        if rate > 0:
            workingValves[valve] = rate
        leadsTo = tokens[9:]
        leadsTo = [item[:-1] for item in leadsTo[:-1]] + [leadsTo[-1]]
        valveMap[valve] = leadsTo
    return valveMap, workingValves

def findMoves(startValve, valveMap):
    rem = deque([startValve])
    distances = {startValve: 0}
    while len(rem) > 0:
        valve = rem.popleft()
        distanceFromStart = distances[valve]
        for nextValve in valveMap[valve]:
            if nextValve in distances:
                continue
            distances[nextValve] = distanceFromStart + 1
            rem.append(nextValve)
    return distances

def mapMoves(valveMap, activeValves):
    activeValveMap = dict()
    for startValve in activeValves:
        moves = findMoves(startValve, valveMap)
        activeValveMap[startValve] = dict()
        for endValve in activeValves:
            if startValve == endValve:
                continue
            activeValveMap[startValve][endValve] = moves[endValve]
    return activeValveMap

def optimizeValves(mm, valves, startvalve, l=30):
    rem = deque([(startvalve, set([startvalve]), 0, 0)])
    bestScore = dict()
    visited = dict()
    while len(rem) > 0:
        valve, openValves, moves, score = rem.popleft()
        frozenValves = frozenset(openValves)
        bestScore[frozenValves] = max(score, bestScore.get(frozenValves, 0))
        key = (valve, frozenValves)
        if key in visited and visited[key][0] <= moves and visited[key][1] >= score:
            continue
        visited[key] = (moves, score)
        for nextValve in valves:
            if nextValve in openValves:
                continue
            newMoves = moves + mm[valve][nextValve] + 1
            if newMoves <= l:
                newScore = score + (valves[nextValve] * (l - newMoves))
                rem.append((nextValve, openValves | {nextValve}, newMoves, newScore))
    return bestScore

def calculateBest(inp, noMoves):
    valveMap, activeValves = parseInp(inp)
    activeValves["AA"] = 0
    activeValveMap = mapMoves(valveMap, activeValves)
    return optimizeValves(activeValveMap, activeValves, "AA", noMoves)


## Examples:
```
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
```

In [4]:
# testing the examples
ex = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""
bestScore = calculateBest(ex, 30)
print("Example 1.1:", max(bestScore.values()))

Example 1.1: 1651


In [5]:
# finding the solution
bestScore = calculateBest(content, 30)
print("Part 1:", max(bestScore.values()))

Part 1: 1737


# Part 2

## Examples:
```
```

In [6]:
# testing the examples
bestScore = calculateBest(ex, 26)
b = max((v1 + v2 for k1, v1 in bestScore.items()
                 for k2, v2 in bestScore.items()
                 if k1 & k2 == frozenset(["AA"])))
print("Part 2:", b)

Part 2: 1707


In [7]:
# finding the solution
bestScore = calculateBest(content, 26)
b = max((v1 + v2 for k1, v1 in bestScore.items()
                 for k2, v2 in bestScore.items()
                 if k1 & k2 == frozenset(["AA"])))
print("Part 2:", b)

Part 2: 2216
